In [30]:
%pip install crewai crewai_tools pandas groq

Note: you may need to restart the kernel to use updated packages.


In [2]:
from crewai import Agent, Task, Crew, LLM
import os
import yaml 
from dotenv import load_dotenv

In [12]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env',override=True)
api_key=os.getenv("GROQ_API_KEY")
serper_key= os.getenv("SERPER_API_KEY")
os.environ["GROQ_API_KEY"]=api_key
os.environ["SERPER_API_KEY"]=serper_key
llm = LLM(model='groq/llama3.1 70B')


In [14]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

In [15]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")


In [16]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool


In [17]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm=llm
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm=llm
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm=llm
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)


In [18]:
# Create the agent
email_content_specialist = Agent(
    config=email_agents_config['email_content_specialist'],
    llm=llm
)

engagement_strategist = Agent(
    config=email_agents_config['engagement_strategist'],
    llm=llm

)

# Create the tasks

email_drafting = Task(
    config=email_tasks_config['email_drafting'],
    agent=email_content_specialist
)
engagement_optimization = Task(
    config=email_tasks_config['engagement_optimization'],
    agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
    agents=[
        email_content_specialist,
        engagement_strategist
    ],
    tasks=[
        email_drafting,
        engagement_optimization
    ],
    verbose = True
)

In [19]:
from crewai import Flow
from crewai.flow.flow import listen, start
import pandas as pd

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):

        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores


    @listen(score_leads)
    def store_leads_score(self, scores):

        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        print('a')
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()


╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: SalesPipeline                                                                                            │
│  ID: 1a01dd6f-d31a-4621-801a-f779396abb46                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [20]:
flow.plot()

Plot saved as crewai_flow.html


In [21]:
emails = await flow.kickoff_async()

🌊 Flow: SalesPipeline
    ID: 1a01dd6f-d31a-4621-801a-f779396abb46
└── 🧠 Starting Flow...

 Flow started with ID: 1a01dd6f-d31a-4621-801a-f779396abb46


🌊 Flow: SalesPipeline
    ID: 1a01dd6f-d31a-4621-801a-f779396abb46
├── 🧠 Starting Flow...
└── 🔄 Running: fetch_leads

🌊 Flow: SalesPipeline
    ID: 1a01dd6f-d31a-4621-801a-f779396abb46
├── Flow Method Step
└── ✅ Completed: fetch_leads

🌊 Flow: SalesPipeline
    ID: 1a01dd6f-d31a-4621-801a-f779396abb46
├── Flow Method Step
├── ✅ Completed: fetch_leads
└── 🔄 Running: score_leads

...

🌊 Flow: SalesPipeline
    ID: 1a01dd6f-d31a-4621-801a-f779396abb46
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
└── ✅ Completed: write_email

🌊 Flow: SalesPipeline
    ID: 1a01dd6f-d31a-4621-801a-f779396abb46
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── 🔄 Running: send_email

🌊 Flow: SalesPipeline
    ID: 1a01dd6f-d31a-4621-801a-f779396abb46
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── ✅ Completed: send_email

✅ Flow Finished: SalesPipeline
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── ✅ Completed: send_email

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: SalesPipeline                                                                                            │
│  ID: 1a01dd6f-d31a-4621-801a-f779396abb46                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [22]:
emails

[CrewOutput(raw="You oversee the development of innovative software solutions at Clearbit, driving business intelligence and growth. Our Multi-Agent Orchestration Platform at CrewAI can help automate complex processes across various verticals by orchestrating AI Agents. \nWith Clearbit's revenue of $41.6 million and a market presence of 7, we believe our solution can streamline operations and enhance efficiency. \nWe'd like to explore how CrewAI can support Clearbit's continued growth and success. \nClick this link to schedule a meeting and discover how our platform can address your specific pain points and interests in agentic automation: [insert scheduling link]\nAlternatively, reply to this email to discuss further or book a call at your convenience. \nTake the first step towards transforming your operations with CrewAI - schedule your meeting now and let's unlock the potential of agentic automation for Clearbit. \n[insert CTA button: Schedule Meeting]", pydantic=None, json_dict=Non